In [1]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F




RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
with open('data/train_sentiment.txt') as f:
    lines  = f.readlines()
    content = []
    sentiment = []
    for line in lines:
        line = line.split('\t')
        content.append(line[1])
        sentiment.append(int(line[2][0]))

In [ ]:
PRE_TRAINED_MODEL_NAME = 'hfl/rbt3'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
class ReviewDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        # numpy data
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return (len(self.reviews))
    
    def __getitem__(self, index):
        review = str(self.reviews[index])
        target = self.targets[index]
        
        encoding = self.tokenizer.encode_plus(review,
                                             add_special_tokens=True,
                                             max_length=self.max_len,
                                             return_token_type_ids=False,
                                             pad_to_max_length=True,
                                             return_attention_mask=True,
                                             return_tensors='pt',
                                             truncation=True)
        return{
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
len(content)

In [ ]:
train_content, test_content,train_sentiment, test_sentiment = train_test_split(content, sentiment,test_size=0.1, random_state=RANDOM_SEED)
test_content, val_content,test_sentiment, val_sentiment = train_test_split(test_content, test_sentiment,test_size=0.1, random_state=RANDOM_SEED)

In [ ]:
MAX_LEN = 150

In [ ]:
def create_data_loader(content, sentiment, tokenizer, max_len, batch_size):
    ds = ReviewDataset(reviews=content,
                      targets=sentiment,
                      tokenizer=tokenizer,
                      max_len=max_len)
    return DataLoader(ds, batch_size=batch_size, num_workers=4)

BATCH_SIZE = 16

train_data_loader = create_data_loader(train_content, train_sentiment, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_content, val_sentiment, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_content, test_sentiment, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
data = next(iter(train_data_loader))
data.keys()

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_class):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_class)
        
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask)
        output = self.drop(pooled_output)
        return self.out(output)

In [ ]:
model = SentimentClassifier(3)
model = model.to(device)

In [ ]:
EPOCHS = 10

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_setps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_setps)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, schduler, n_examples):
    model = model.train()
    
    losses = []
    correct_predictions = 0
    
    for d in data_loader:
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        targets = d['targets'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        
        correct_predictions += torch.sum(preds==targets)
        losses.append(loss.item())
        
        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    mode = model.eval()
    
    losses = []
    correct_predictions = 0
    
    with torch.no_grad():
        for d in data_loader:
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            targets = d['targets'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _,  preds = torch.max(outputs, dim=1)
            
            loss = loss_fn(outputs, targets)
            
            correct_predictions += torch.sum(preds==targets)
            losses.append(loss.item())
            
    return correct_predictions.double() / n_examples, np.mean(losses)                

In [ ]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1}/{EPOCHS}')
    print('-'*10)
    
    train_acc, train_loss = train_epoch(model, train_data_loader, loss_fn, optimizer, device, scheduler, len(train_content))
    
    print(f'Train loss {train_loss}, accuracy {train_acc}')
    
    val_acc, val_loss = eval_model(model, val_data_loader, loss_fn, device, len(df_val))
    
    print(f'Val loss {val_loss}, accuracy {val_acc}')
    print()
    
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['vla_acc'].append(val_acc)
    history['vla_loss'].append(val_loss)
    
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        torch.save(model.state_dict, 'bset_mode_state.bin')

In [ ]:
model = SentimentClassifier(3)

model.load_state_dict(torch.load('best_mode_state.bin', map_location=torch.device('cpu')))
model = model.to(device)

In [ ]:
test_acc, _ = eval_model(model, test_data_loader, loss_fn, device, len(test_content))

test_acc.item()

In [ ]:
def get_predictions(model, data_loader):
    model = model.eval()
    
    review_texts = []
    predictions = []
    prediction_probs =[]
    real_values = []
    
    with torch.no_grad():
        for d in data_loader:
            texts = d['review_text']
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            targets = d['targets'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            
            review_texts.extend(texts)
            predictions.extend(preds)
            prediction_probs.extend(outputs)
            real_values.extend(targets)
            
    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    return review_texts, predictions, prediction_probs, real_values
            

In [ ]:
review_text = "房间的卫生间较差，不尽人意。总的感觉不是很好。但是个人用品的卫生还是很规范的，整洁而且包装的很完整。"

In [ ]:
encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)

In [3]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_class):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_class)
        
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask)
        output = self.drop(pooled_output)
        return self.out(output)
    
PRE_TRAINED_MODEL_NAME = 'hfl/rbt3'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
class_names = ['中立', '正面', '反面']

MAX_LEN = 150

model = SentimentClassifier(len(class_names))

model.load_state_dict(torch.load('best_mode_state.bin', map_location=torch.device('cpu')))
model = model.to(device)

review_text = "房间的卫生间较差，不尽人意。总的感觉不是很好。但是个人用品的卫生还是很规范的，整洁而且包装的很完整。"

encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)

input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)

output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)

print(f'Review text: {review_text}')
print(f'Sentiment  : {class_names[prediction]}')
print(prediction)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Review text: 房间的卫生间较差，不尽人意。总的感觉不是很好。但是个人用品的卫生还是很规范的，整洁而且包装的很完整。
Sentiment  : 中立
tensor([0])


/Users/vito/opt/anaconda3/envs/ml/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
